# Computational Methods in Economics

## Lecture 5b - Numerical Optimization and Data Analysis

In [1]:
# Author: Alex Schmitt (schmitt@ifo.de)

import datetime
print('Last update: ' + str(datetime.datetime.today()))

Last update: 2018-12-05 14:08:07.928452


### Preliminaries

#### Import Modules

In [2]:
import numpy as np
import pandas as pd

np.set_printoptions(precision=4, suppress = True)

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn

import statsmodels.api as sm

import scipy.optimize
import scipy.linalg

## This Lecture

- [Introduction](#introduction)
- [The Scikit-Learn Package](#scikit)
- [Notation and Terminology](#not)
- [Linear Least-Squares Problems](#lls)
- [Gradient Descent](#grad)
- [Nonlinear LS: LASSO](#lasso)
- [Classification: Logistic Regression](#logistic)

--------------------------------------------------------------------------------------------------------------------------------
<a id = 'introduction'></a>

### Introduction

In the previous lecture we have discussed how to solve general optimization problems numerically. In this class, we are going to look at particular optimization problems that (among other applications) arise frequently in the context of data analysis.

In addition, we also introduce more functionality for Python, in particular the scikit-learn package.

Scikit-learn is most commonly used in the context of "data science" and "machine learning". An additional goal of this class is to give a short introduction in particular to machine learning (also called "statistical learning"), and show how it differs (and what it has in common) with more standard (from an economist's point of view) approaches to data analysis in econometrics.

It goes without saying that given the complexity of the topic and the limited amount of time, we will not be able to go in much depth. That said, the idea is to introduce some core ideas and terminology, and leave the rest to more specialized courses.

--------------------------------------------------------------------------------------------------------------------------------
<a id = 'scikit'></a>

### The Scikit-Learn Package

Scikit-learn (**sklearn**) is a very popular package for data analysis in  general and machine learning in particular. It builds on the packages we have already seen - NumPy, SciPy and Matplotlib - and provides a lot of additional functionality (of which we will only scratch the surface).

Its documentation and an extensive amount of tutorials and other material can be found here: https://scikit-learn.org/stable/index.html.

Similar to SciPy, to use Scikit-Learn, we import individual subpackages rather than the whole package:

In [3]:
from sklearn.datasets import load_boston
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

For now, just focus on the first statement in the cell above. Scikit-Learn includes a few "toy datasets" (as well as easy access to some larger "real-world" datasets), one of which we will use as a workhorse example below. It contains data on house prices and features in the Boston area. 

We load the dataset - which comes in the form of what is essentially a dictionary - and display its **DESCR** key, which contains a description.

In [4]:
boston = load_boston()
print( boston.DESCR )

Boston House Prices dataset

Notes
------
Data Set Characteristics:  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
      

We can check all key-value pairs in the dictionary by running the **keys()** method:

In [5]:
print( boston.keys() )

dict_keys(['data', 'target', 'feature_names', 'DESCR'])


The data is contained in the **data** and the **target** field. We can check that the number of data points in **data** are consistent with the description:

In [6]:
print("Data shape: {}".format(boston.data.shape))

Data shape: (506, 13)


--------------------------------------------------------------------------------------------------------------------------------
<a id = 'not'></a>

### Notation and Terminology

In the following, we use the notation that is standard in data analysis, in particular econometrics and data science. While this makes it consistent with other courses that you might have taken in the past, the notation will be different compared to the previous lectures in this class.


In particular, let $(y_i, \mathbf{x}_i)$ be a given tuple of data corresponding to observation $i$. 

$y_i$ is a scalar and referred to as a dependent variable, a *label* (if it comes from a discrete set of values) or *target variable* (if it is a continuous variable).

$\mathbf{x}_i = [x_{i1}, ... , x_{in}]^T \in \mathbb{R}^n$ is a vector of dimension $n$. Each element of $\mathbf{x}_i$ is referred to as an independent variable, a *regressor* or a *feature*.

For a data set consisting of $m$ observations, $\mathbf{y} = [y_{1}, ... , y_{m}]^T  \in \mathbb{R}^m$ is a vector of outcome variables, while

\begin{equation}
    \mathbf{X} =  \left[ \begin{array}{c}
    \mathbf{x}_1^T  \\
    \vdots \\
    \mathbf{x}_m^T 
    \end{array} \right] =  
    \left[ \begin{array}{cccc}
    x_{11} & x_{12} & ... & x_{1n}  \\
    x_{21} & x_{22} & ... & x_{2n}  \\
    \vdots \\
    x_{m1} & x_{m2} & ... & x_{mn}  \\
    \end{array} \right] 
\end{equation}

is an *m*-by-*n* matrix containing all realizations of the feature variables. It is also referred to as the regression or feature matrix. 

In the context of our example dataset above, we define **y** as the vector of target variables (in **boston.target**) and **X** as the feature matrix containing the regressors in **boston.data**:

In [7]:
y = boston.target
X = boston.data

In [8]:
## display first five observations
print( X[:5, :])

[[  0.0063  18.       2.31     0.       0.538    6.575   65.2      4.09
    1.     296.      15.3    396.9      4.98  ]
 [  0.0273   0.       7.07     0.       0.469    6.421   78.9      4.9671
    2.     242.      17.8    396.9      9.14  ]
 [  0.0273   0.       7.07     0.       0.469    7.185   61.1      4.9671
    2.     242.      17.8    392.83     4.03  ]
 [  0.0324   0.       2.18     0.       0.458    6.998   45.8      6.0622
    3.     222.      18.7    394.63     2.94  ]
 [  0.0691   0.       2.18     0.       0.458    7.147   54.2      6.0622
    3.     222.      18.7    396.9      5.33  ]]


##### Hypothesis and Residual Function

On a very general level, when analyzing data, we compare the realizations of the target variable in the sample, $\mathbf{y}$, with a vector $[\hat{y}_{1}, ... , \hat{y}_{m}]^T$ of *predicted* outcomes. 

The predicted value of the target variable for observation $i$ is given by a function $h$, which (in machine learning terminology) is called the *hypothesis*. 

More precisely, the function $\hat{y}_{i} = h(\mathbf{x}_i; \mathbf{b})$ returns the prediction for the target variable $y_i$ given the feature vector $\mathbf{x}_i$ and a vector of *parameters* $\mathbf{b}$. 

No matter what the purpose of an empirical analysis is, e.g. causal estimation (in econometrics) or prediction (in statistical/machine learning), it usually requires determining the parameter values given a data set, $(\mathbf{y}, \mathbf{X})$. This step is done by setting it as a minimization problem.

That is, we choose $\mathbf{b}$ by minimizing a measure of the difference between the observed values of the outcome variable and its predicted values when using the hypothesis function. For a single observation, this difference is called the *residual (function)*:

\begin{equation}
    h(\mathbf{x}_i; \mathbf{b})- y_i \equiv r_i(\mathbf{b}; y_i, \mathbf{x}_i) = r_i(\mathbf{b})
\end{equation}

Note the following:

- We have defined $h$ as a function of the data $\mathbf{x}_i$, given parameter values $\mathbf{b}$. In contrast, we write the residual as a function of $\mathbf{b}$. Of course, $r_i$ also depends on the data, but we usually omit these as arguments.

- Related to this, we have different residual functions $r_i$ for every observation $i$.

- Every $r_i$ is a scalar function, with $r_i: \mathbb{R}^n \rightarrow \mathbb{R}$.

Finally, let $\mathbf{r}$ denote the vector of residuals (i.e. a vector-valued function $\mathbb{R}^n \rightarrow \mathbb{R}^m$) across the sample:

\begin{equation}
    \mathbf{r}(\mathbf{b})  =  \left[ \begin{array}{c}
    r_1(\mathbf{b})  \\
    \vdots \\
    r_m(\mathbf{b}) 
    \end{array} \right] 
\end{equation}

#### Least-Squares Minimization

There are different alternatives with respect to how to set up the objective function $L(\mathbf{b})$ in the minimization problem to find the parameter values $\mathbf{b}$:

\begin{equation}
    \min_{\mathbf{b}} L(\mathbf{b})
\end{equation}

$L$ is also referred to as the *loss function* or *cost function*.

#####  $\mathcal{l}_\infty$-norm

Define $L$ as 

\begin{equation}
    L(\mathbf{b}) = \left| \left| \mathbf{r}(\mathbf{b}) \right| \right|_{\infty} = \max_{i = 1,...,m} \left| r_i(\mathbf{b}) \right|
\end{equation}

Hence, the loss function is determined by the data point with the largest deviation (in absolute value) from its predicted value.

#####  $\mathcal{l}_1$-norm

Define $L$ as 

\begin{equation}
    L(\mathbf{b}) = \left| \left| \mathbf{r}(\mathbf{b}) \right| \right|_{1} = \sum_{i = 1}^m \left| r_i(\mathbf{b}) \right|
\end{equation}

Hence, the loss function is the sum of absolute values of the residuals.

#####  $\mathcal{l}_2$/Euclidean norm

Define $L$ as 

\begin{equation}
    L(\mathbf{b}) = \frac{1}{2} \left(\ \left| \left|\ \mathbf{r}(\mathbf{b}) \right| \right|_{2}\ \right)^2 = \frac{1}{2}\sum_{i = 1}^m \left[r_i(\mathbf{b})\right]^2
\end{equation}


As a side note, recall that for a vector $\mathbf{v} \in \mathbb{R}^n$, the Euclidean norm is defined as 

\begin{equation}
     \left| \left|\ \mathbf{v}\ \right| \right|_2 = \left| \left|\ \mathbf{v}\ \right| \right| = \sqrt{ \sum_{j = 1}^n v_i^2} = \sqrt{ \mathbf{v}^T \mathbf{v} }
\end{equation}

Above, we use one half of the square of the norm; of course, this does not affect the result of the minimization. 

Hence, the loss function is the sum of squares of the residuals. Minimizing this function is the by far most common method for finding $\mathbf{b}$.

In other words, we set up the problem of determining $\mathbf{b}$ as a *least-squares (LS) problem*.

LS problems are an extremely important class of unconstrained optimization problems. Compared to a generic minimization problem, as discussed in the previous lecture, the special form of the objective function in 

\begin{equation}
   \min_{\mathbf{b}} L(\mathbf{b}) = \min_{\mathbf{b}} \frac{1}{2}\sum_{i = 1}^m \left[r_i(\mathbf{b})\right]^2
\end{equation}

makes LS problems easier to solve than more general problems (see below). 

Before moving on, let's recap the different types of functions defined above:
- the hypothesis function $h(\mathbf{x}_i; \mathbf{b})$ gives a prediction of the target value for data point $\mathbf{x}_i$;
- the difference between the actual and predicted value gives rise to the residual function $r_i(\mathbf{b})$;
- in LS, the loss function is defined as the sum of squared residuals; minimizing it determines $\mathbf{b}$. 


Note that the LS method imposes a functional form on $L$, but does not prescribe what $h$ (and hence $r$) looks like. If $h$ is linear in the parameters $\mathbf{b}$, the problem becomes the familiar LS linear regression framework that you are familiar with.

It is important to understand though that this is one particular class of LS problems. In other words, the use LS is not restricted to linear hypothesis functions. 

--------------------------------------------------------------------------------------------------------------------------------
<a id = 'lls'></a>

### Linear Least-Squares Problems

In linear LS problems, the hypothesis function is *linear in the parameters* $\mathbf{b}$ (but not necessarily linear in the features $\mathbf{x}$!).

Hence,

\begin{equation}
    \hat{y}_i = h(\mathbf{x}_i; \mathbf{b}) = b_1 x_{i1} + b_2 x_{i2} + ... + b_{n} x_{in} = \sum_{j = 1}^n b_j x_{ij} = \mathbf{x}_i^T \mathbf{b}. 
\end{equation}


Note that while this notation makes it look like all the $x_{ij}$'s are distinct features, some can be a function of the others, e.g.

\begin{equation}
    x_{i2} = x_{i1}^2. 
\end{equation}

For the whole sample, we have

\begin{equation}
    \hat{\mathbf{y}}  = \left[ \begin{array}{c}
    \hat{y}_1\\
    \vdots \\
    \hat{y}_m
    \end{array} \right]= \left[ \begin{array}{c}
    \mathbf{x}_1^T \mathbf{b} \\
    \vdots \\
    \mathbf{x}_m^T \mathbf{b}
    \end{array} \right] = \mathbf{X} \mathbf{b}
\end{equation}


Hence, in matrix notation, we can express the loss function in a number of equivalent ways:

\begin{equation}
    L(\mathbf{b}) = \frac{1}{2}\sum_{i = 1}^m \left[\hat{y}_i - y_i\right]^2 = \frac{1}{2} \ \left| \left|\ \hat{\mathbf{y}} - \mathbf{y}\ \right| \right|^2 = \frac{1}{2} \left| \left|\ \mathbf{X} \mathbf{b} - \mathbf{y}\ \right| \right|^2 = \frac{1}{2} \left| \left|\ \begin{array}{c}
    \mathbf{x}_1^T \mathbf{b} - y_1 \\
    \vdots \\
    \mathbf{x}_m^T \mathbf{b} - y_m
    \end{array}
    \ \right| \right|^2 = \frac{1}{2} (\mathbf{X} \mathbf{b} - \mathbf{y})^T (\mathbf{X} \mathbf{b} - \mathbf{y})
\end{equation}

When implementing the loss function, the last definition is computationally most efficient:

In [9]:
## define loss function
def loss(b, X):
    predicted_values = X @ b
    return ( predicted_values - y ).T @ ( predicted_values - y ) / m

#### Gradient

The derivative of the loss function w.r.t. to $b_j$ is:

\begin{equation}
    \frac{\partial L}{\partial b_j} = \sum_{i = 1}^m \left[ \hat{y}_i - y_i \right] x_{ij} = \mathbf{X}_j^T \cdot (\hat{\mathbf{y} } - \mathbf{y}),
\end{equation}

where $\mathbf{X}_j$ denotes the $j$th column in the regression matrix. 

Hence, the gradient of the loss function is given by:

\begin{equation}
    \nabla L = \left[ 
        \begin{array}{c}
        \frac{\partial L}{\partial b_1} \\
        \vdots \\
        \frac{\partial L}{\partial b_m}
        \end{array}
    \right] =  \mathbf{X}^T \cdot(\hat{\mathbf{y} } - \mathbf{y}) = \mathbf{X}^T \cdot(\mathbf{X} \mathbf{b} - \mathbf{y}) 
\end{equation}.

Moreover, it is straightforward to see that the Hessian of the loss function is:

\begin{equation}
    \nabla^2 L = \left[ 
        \begin{array}{ccc}
        \frac{\partial^2 L}{\partial b_1 \partial b_1} & ... & \frac{\partial^2 L}{\partial b_1 \partial b_n} \\
        \vdots & \ddots & \vdots \\
        \frac{\partial^2 L}{\partial b_n \partial b_1} & ... & \frac{\partial^2 L}{\partial b_n \partial b_n}
        \end{array}
    \right]  = \mathbf{X}^T \mathbf{X}
\end{equation}

Two things are noteworthy here:

- the Hessian depends solely on the feature matrix $\mathbf{X}$ and is independent of $\mathbf{b}$; hence, it needs to be computed only once;
- it is positive definite, implying that the loss function is convex. Hence, any $\mathbf{b}^*$ such that $\nabla L(\mathbf{b}^*) = 0$ is the *global* minimizer of $L$. 

It should be emphasized that these properties are particular for linear LS problems, and do not necessarily hold for non-linear problems (and of course not for general minimization problems either). 

As a side note, $\mathbf{X}^T \mathbf{X}$ being positive definite may not be obvious at first glance. However, you can show that for an arbitrary $m$-by-$n$ matrix $A$ with $m \ge n$ and rank $n$ - i.e., that has linearly independent columns - the matrix product $A^T A$ is positive definite. This will be shown in this week's problem set. 

Moreover, by construction, $A^T A$ is also symmetric. 

#### Normal Equations

We can use the gradient directly for computing a minimum of the loss function, by setting it equal to zero: $\nabla L  = 0$, or 

\begin{equation}
     \mathbf{X}^T \cdot(\hat{\mathbf{y} } - \mathbf{y}) = \mathbf{X}^T \cdot(\mathbf{X} \mathbf{b} - \mathbf{y}) \overset{!}{=} 0
\end{equation}

Hence, $\mathbf{b}^*$ must satisfy the following linear system of equations:

\begin{equation}
     (\mathbf{X}^T \mathbf{X}) \mathbf{b} = \mathbf{X}^T \mathbf{y}
\end{equation}

These are called the *normal equations*. Solving them gives a closed-form solution for $\mathbf{b}^*$.

Solving for $\mathbf{b}^*$ by using the inverse of $\mathbf{X}^T \mathbf{X}$ gives the familiar formula for the OLS estimator:

\begin{equation}
     \mathbf{b}^* = \left(\mathbf{X}^T \mathbf{X}\right)^{-1} \mathbf{X}^T \mathbf{y}
\end{equation}

In practice, statistical programs do not use the inverse to solve the normal equations. Instead, they rely on methods that follow the same idea as LU factorization and triangular back- and forward substitution that we have encountered in lecture 3.

Instead of LU factorization, however, the fact that $\mathbf{X}^T \mathbf{X}$ is a symmetric and positive definite matrix, as established above, allows to use *Cholesky factorization*, which requires only half the number operations compared to LU. 

In more detail, for any symmetric positive definite $n$-by-$n$ matrix $A$, Cholesky factorization produces the following decomposition:

\begin{equation}
    A = U^T U,
\end{equation}

where $U$ is an $n$-by-$n$ upper triangular matrix with positive diagonal elements. Once we have obtained $U$, we can easily apply triangular substitution.

As a side note, Cholesky factorization can be used to check if a symmetric matrix is positive definite. If the algorithm returns a matrix $U$ where the diagonal elements are well-defined and positive, positive definiteness is verified.

In SciPy, we have a function **scipy.linalg.cholesky** that returns $U$. As an example, let's use the feature matrix **X** from our data set, perform Cholesky factorization and check the diagonal elements of **U**.

In [10]:
A = X.T @ X
print(A.shape)

(13, 13)


In [11]:
U = scipy.linalg.cholesky(A)
print( np.diag( U ) )

[ 209.4204  583.1158  247.9859    5.6858    4.2375   26.1879  402.8817
   31.232   130.9807 1295.7138   49.1592 1854.1449   98.1221]


For the sake of completeness, if you try to apply Cholesky factorization on a matrix that is not positive definite, SciPy raises an error:

In [12]:
A = np.array([[1., 2.], 
              [2., 4.], 
              [3., 6.]])

print( np.linalg.matrix_rank(A) )
## this line will throw an error, since A has rank 1 < n and hence A.T @ A is not positive definite!
# print( scipy.linalg.cholesky(A.T @ A) )


1


Going back on how to solve the normal equations for $\mathbf{b}^*$, an important disadvantage of Cholesky factorization is that the condition number of $\mathbf{X}^T \mathbf{X}$ is the square of the condition number of $\mathbf{X}$. 

Recall that the higher the condition number, the less well-conditioned a matrix is, and the higher the numerical error in the computed solution.

Hence, the Cholesky-based method can result in less accurate solutions than when using methods that avoid a squaring of the condition number. When $\mathbf{X}$ is ill-conditioned, Cholesky factorization may even break down.

In addition, Cholesky will also not work if $\mathbf{X}$ is "rank-deficient", i.e. has rank less than $n$.

That's why in practice, solving the normal equations is often done by means of factorization of $\mathbf{X}$ rather than $\mathbf{X}^T \mathbf{X}$. Among the alternative approaches are *QR factorization* and *singular-value decomposition* (SVD). 

While potentially more computationally expensive, both methods approach are more robust and reliable than Cholesky, since avoid squaring the condition number. SVD also allows for rank-deficient matrices. 

While we will not go into more detail here, note that the SVD-approach is the default in SciPy's **scipy.linalg.lstsq** function for solving linear least-squares problems.  

#### Solving Linear LS Problems in Python

In Python, there are several alternative packages and functions for solving linear LS problems. We have already seen that the **statsmodels** package has a OLS routine, which is very convenient in connection with Pandas (see the tutorial on Pandas).

As mentioned above, SciPy has a function **scipy.linalg.lstsq** that returns the parameter vector $\mathbf{b}^*$ for a given feature matrix $\mathbf{X}$ and target vector $\mathbf{y}$, along with some additional information as a tuple. 

Before running the function, we add a column to our feature matrix that consists of ones. This takes care of the intercept in the linear regression problem. In other words, we assume that $x_{i1} = 1$ for all observations. 

In [13]:
X_1 = np.column_stack( (np.ones( X.shape[0] ), X ) )
print(X_1[:5, :5])

[[ 1.      0.0063 18.      2.31    0.    ]
 [ 1.      0.0273  0.      7.07    0.    ]
 [ 1.      0.0273  0.      7.07    0.    ]
 [ 1.      0.0324  0.      2.18    0.    ]
 [ 1.      0.0691  0.      2.18    0.    ]]


In [14]:
res = scipy.linalg.lstsq(X_1, y)
## print parameter values
print( res[0])

[ 36.4911  -0.1072   0.0464   0.0209   2.6886 -17.7958   3.8048   0.0008
  -1.4758   0.3057  -0.0123  -0.9535   0.0094  -0.5255]


Alternatively, we can set up the linear LS problem using scikit-learn's **LinearRegression()** class (which we have imported above). Note that this a "wrapper" around **scipy.linalg.lstsq**, i.e. it uses this function for the minimization, but then presents the output in a different, nicer way.

Note also that we do not have to add a column of ones to the feature matrix; this is done by default.

In [15]:
lr = LinearRegression()
lr.fit(X, y)
print("Intercept: {}".format(lr.intercept_) )
print("Coefficients: {}".format(lr.coef_) )
## store results for later comparison
b_lls = np.insert( lr.coef_, 0, lr.intercept_)

Intercept: 36.49110328036103
Coefficients: [ -0.1072   0.0464   0.0209   2.6886 -17.7958   3.8048   0.0008  -1.4758
   0.3057  -0.0123  -0.9535   0.0094  -0.5255]


In [16]:
## for comparison: compute b by hand, using the inverse
b = np.linalg.inv((X_1.T @ X_1)) @ X_1.T @ y
print(b)

[ 36.4911  -0.1072   0.0464   0.0209   2.6886 -17.7958   3.8048   0.0008
  -1.4758   0.3057  -0.0123  -0.9535   0.0094  -0.5255]


Note that this syntax is typical of scikit-learn, and we will see more examples below. We start by initiating an instance of a **LinearRegression()** class, which does not take any arguments. Solving the linear LS problem happens in the second line where we use the **fit** method, and give it the feature matrix and the target vector as inputs.

The parameter values can then be found in the **coef_** and **intercept_** attribute. 

Finally, let's emphasize once more that the closed-form solution coming from the normal equations exists only in the case of linear LS. For nonlinear LS problem, this direct approach is not (typically) available.

Moreover, there are LS problems where finding $\mathbf{b}^*$ by solving the normal equations is not the most efficient. In all these cases, we will have to resort to iterative methods once again. 

--------------------------------------------------------------------------------------------------------------------------------
<a id = 'grad'></a>

### Quasi-Newton Methods: Gradient Descent

Before moving on to nonlinear LS problems, we will get to know one more algorithm for numerical optimization, *gradient descent* or *steepest descent*. 

While this is essentially a simple variant of Quasi-Newton methods and can be used to solve general optimization problems, its use is particularly common in the context of machine learning. 

As a quick recap, we have introduced the class of line search method. Recall that line search is an iterative procedure, generating a sequence of iterates $\{ x^{(k)} \}$ that are aimed to converge towards the minimum $x^*$. The update rule is

\begin{equation}
    \mathbf{x}^{(k+1)} = \mathbf{x}^{(k)} + \mathbf{s}^{(k)} = \mathbf{x}^{(k)} + \alpha^{(k)} \mathbf{d}^{(k)}
\end{equation}

with *search direction* $\mathbf{d}^{(k)}$ and *distance* $\alpha^{(k)}$, i.e. the length of the *step* $\mathbf{s}^{(k)}$.

On a general level, line search algorithms take the direction $\mathbf{d}^{(k)}$ as given and then choose $\alpha$ by solving the following problem:

\begin{equation}
    \min_{\alpha > 0} f(\mathbf{x}^{(k)} + \alpha \mathbf{d}^{(k)})
\end{equation}


We have also noted that different variants of line search methods differ in their choice of the search direction (and some also fix $\alpha$), and hence with respect to how to determine $\mathbf{s}^{(k)} = \alpha^{(k)} \mathbf{d}^{(k)}$. For example, Newton's method uses the following step, with a fixed $\alpha$:

\begin{equation}
 \mathbf{s}^{(k)} =  - H(\mathbf{x}^{(k)}) ^{-1}\nabla  f(\mathbf{x}^{(k)}) 
\end{equation}

Other than this, they all follow the same iterative procedure (which we had implemented in the function **my_opt**).

The gradient descent algorithm uses the gradient as the search direction:

\begin{equation}
  \mathbf{d}^{(k)} = - \nabla  f(\mathbf{x}^{(k)})
\end{equation}

and then looks for the optimal $\alpha^{(k)}$, for example by using Armijo search and backtracking. 

Hence, we can interpret gradient descent as a simple Quasi-Newton method, where the Hessian in the Newton-Raphson update rule is *approximated by the identity matrix*. 

Gradient descent is very intuitive: in order to minimize, we move in the direction of a local minimum. The main problem of gradient descent is that convergence is in general very slow, in particular for objective function that are highly non-linear. 

The Rosenbrock function is good example: even when using an optimal $\alpha$ in every iteration, it can take thousands of iterations to get to the minimum. While gradient descent gets down to the Rosenbrock valley very quickly, it then makes very slow progress towards to the minimum (compare http://www.benfrederickson.com/numerical-optimization/ for a visualization).

#### Gradient Descent for Linear Regression

However, gradient descent is popular in applications where the objective is more regular (say, quadratic, and hence convex) and hence the gradient is linear. 

The obvious example is linear LS problems, for whcih gradient descent can be used as an alternative to the standard approach based on normal equations. 

Related to that, gradient descent is very important and heavily used in the context of machine learning, where you usually have a very large number of observations and explanatory variables/features (where the so-called "normal equations" approach may not be applicable).

Going back to the notation used in this lecture, gradient descent uses the update rule

\begin{equation}
    \mathbf{b}^{(k+1)} = \mathbf{b}^{(k)} + \alpha^{(k)} \mathbf{d}^{(k)} = \mathbf{b}^{(k)} - \alpha^{(k)} \nabla  L(\mathbf{b}^{(k)} ) = \mathbf{b}^{(k)} - \alpha^{(k)} \left[ \mathbf{X}^T \cdot(\mathbf{X} \mathbf{b}^{(k)} - \mathbf{y}) \right]
\end{equation}

where we have used the gradient for linear LS problems as derived above. 

In other words, the search direction used by gradient descent is the gradient of the objective function (here the loss function) at the current guess for $\mathbf{b}^{(k)}$.

Without matrix notation, one can compute the coefficient sequentially:
\begin{equation}
    b_j^{(k+1)} = b_j^{(k)} + \alpha^{(k)} \frac{\partial L(\mathbf{b})}{\partial b_j} = b_j^{(k)} - \alpha^{(k)} \sum_{i = 1}^m \left[ \hat{y}_i - y_i \right] = b_j^{(k)} - \alpha^{(k)} \mathbf{X}_j^T \cdot (\mathbf{X} \mathbf{b}^{(k)} - \mathbf{y})
\end{equation}

#### Numerical Implementation

For general optimization problems, we have used the function **my_opt** and defined the update rule corresponding to the specific algorithm in a separate function. Here, we combine those two steps in a single function, called **gd_linear**. Note that we use the precise gradient of the loss function.  

In [17]:
def gd_linear(b, X, y, alpha, show_it = False, maxit = 50000, eps = 1e-8):
    """
    Implements the iterative procedure for gradient descent in the context of linear regression. Inputs are 
    -> b: initial guess for the minimizing coefficient vector
    -> X: the m-by-n regression matrix, with each row containing the n features for one observation
    -> y: the m-by-1 vector of target values 
    -> alpha: a scalar indicating the step size from b(k) to b(k+1)
    """
    ## input check
    m = len(y)
    y.shape = (m, 1)
    assert X.shape[0] == m; "Dimension mismatch between y and X" 
    
    b.shape = (X.shape[1], 1)
    
    dist = 1
    it = 0
    
    while dist > eps and it < maxit:
        it += 1
    
        s = -alpha * X.T @ ( X @ b - y )

        dist = np.linalg.norm(s) / (1 + np.linalg.norm(b))

        b = b + s
    
    if show_it:
        print("Gradient descent has converged in {} iterations".format(it))
    
    return b


Note that since the loss function is convex, we know that convergence to a minimum is guaranteed for a sufficiently small **alpha**. Hence, we do not need to check whether the outcome of our iterative procedure is a minimum. 

##### Feature Scaling

While the idea of the GD algorithm is straightforward, there are some issues to consider when running it in practice. GD may run into problems when the features are not on a similar scale/magnitude. 

In the Boston data set, for example, we have some features that are measured on a scale between approximately 3 and 9 (average number of rooms per dwelling), while others are on a scale from 0 to 400. We can see this by looking at the min and max values for each columns. 

In [18]:
X.max(axis = 0)

array([ 88.9762, 100.    ,  27.74  ,   1.    ,   0.871 ,   8.78  ,
       100.    ,  12.1265,  24.    , 711.    ,  22.    , 396.9   ,
        37.97  ])

In [19]:
X.min(axis = 0)

array([  0.0063,   0.    ,   0.46  ,   0.    ,   0.385 ,   3.561 ,
         2.9   ,   1.1296,   1.    , 187.    ,  12.6   ,   0.32  ,
         1.73  ])

A simplest remedy against this is feature scaling, i.e. bringing the features onto the same scale. There are different ways to do this, but the most common is *standardization*:

\begin{equation}
    x_{ij}^{std} = \frac{x_{ij} - \mu_j}{\sigma_j},
\end{equation}

where $\mu_j$ is the (sample) mean of one feature (i.e. for one column of $\mathbf{X}$) and $\sigma_j$ is the corresponding standard deviation.

Using standardization, features are centered around 0 and have a standard deviation of 1; their distribution (across a column) is normal.

We can easily implement standardization:

In [20]:
## standardize X
(X - X.mean(axis = 0)) / X.std(axis = 0)

array([[-0.4177,  0.2848, -1.2879, ..., -1.459 ,  0.4411, -1.0756],
       [-0.4153, -0.4877, -0.5934, ..., -0.3031,  0.4411, -0.4924],
       [-0.4153, -0.4877, -0.5934, ..., -0.3031,  0.3964, -1.2087],
       ...,
       [-0.4114, -0.4877,  0.1157, ...,  1.1765,  0.4411, -0.983 ],
       [-0.4057, -0.4877,  0.1157, ...,  1.1765,  0.4032, -0.8653],
       [-0.4129, -0.4877,  0.1157, ...,  1.1765,  0.4411, -0.6691]])

Scikit-learn has a function **preprocessing.scale** that does precisely that:

In [21]:
X_sc = preprocessing.scale(X, axis = 0)

In [22]:
## check if the scale function gives the same result as manual standardization
(X_sc == (X - X.mean(axis = 0)) / X.std(axis = 0)).all()

True

We can verify that each column of **X_sc** has mean zero and standard deviation equal to 1:

In [23]:
X_sc.mean(axis = 0)

array([ 0., -0., -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0.,  0., -0.])

In [24]:
X_sc.std(axis = 0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

##### Choosing the Learning Rate 

As outlined last time, "proper" line search algorithms look for the optimal learning rate (see above). In simple implementations of GD, the learning rate is kept fixed. 

The question then arises what value to choose for the learning rate. In general, for a sufficiently small learning rate, you get convergence, and hence a smaller function value, in every iteration. However, the smaller the learning rate, the slower is convergence.

Conversely, if the learning rate is too large, the algorithm may not converge. 

In practice, one would start with a very small learning rate and then increase it, for example by a factor of 3. That is, say you start with *1e-5*, *3e-5*, *1e-4*, ... , until you no longer get convergence.

##### Running GD

Below, we add a column of 1's to the standardized feature matrix **X_sc** and then run our **gd_linear** function defined above. We use a small learning rate of *1e-4*. As initial guess, for simplicity,  we choose a vector of length $n$, filled with 0.1.

Note that the standardization happens before adding the intercept term; moreover, we use scikit-learn's **preprocessing.add_dummy_feature** function for adding the 1-column.  

In [25]:
X_sc = preprocessing.scale(X, axis = 0)
## add column of 1s for intercept term
X_sc = preprocessing.add_dummy_feature(X_sc, 1)
## alternatively (as above)
# X_sc = np.column_stack( (np.ones( X_sc.shape[0] ), X_sc ) )

gd_linear(0.1 * np.ones(X_sc.shape[1]), X_sc, y, 0.0003, maxit = 10000, show_it = True)

Gradient descent has converged in 1218 iterations


array([[22.5328],
       [-0.9204],
       [ 1.081 ],
       [ 0.143 ],
       [ 0.6822],
       [-2.0601],
       [ 2.6706],
       [ 0.0211],
       [-3.1044],
       [ 2.6588],
       [-2.0759],
       [-2.0622],
       [ 0.8566],
       [-3.7487]])

Note that these parameter values are different from the ones that we have computed above using the normal equations. This is due to the fact that we have not use the standardized feature matrix above. Repeating the computation with **X_sc** gives the same result. 

Note that since **X_sc** already contains a 1-column, we tell the algorithm not to add one automatically, by setting **fit_intercept = False**.

In [26]:
lr = LinearRegression(fit_intercept = False)
lr.fit(X_sc, y)
print("Coefficients: {}".format(lr.coef_) )

Coefficients: [[22.5328 -0.9204  1.081   0.143   0.6822 -2.0601  2.6706  0.0211 -3.1044
   2.6588 -2.0759 -2.0622  0.8566 -3.7487]]
